In [1]:
import pandas as pd, numpy as np, pyodbc
pd.options.display.float_format = '{:,.4f}'.format
from sklearn.ensemble import RandomForestRegressor
from sktime.forecasting.theta import ThetaForecaster
import plotly.express as px; from plotly import graph_objects as go
import datetime as dt; ts = dt.datetime.now(); ts_str = ts.strftime("%Y_%m_%d_%H_%M_%S"); print(ts_str)

2020_09_10_08_49_04


In [2]:
%%time
con = (
    r'DRIVER=ODBC Driver 17 for SQL Server;'
    r'SERVER=172.31.228.90;'
    r'DATABASE=MPlanProd;'
    r'Trusted_Connection=no;'
    r'Uid=DataEng;'
    r'Pwd=Data@2020;'
)
sql = 'SELECT * FROM [MPlanProd].[dbo].[View_Transaction_SFT_6049]'
cxn = pyodbc.connect(con)
df1 = pd.read_sql(sql, cxn)
print(df1.shape, ', '.join(df1.columns))

(284702, 19) SAPCode, SiteName, WorkCentreName, Sloc, Day, Time, SupplierName, PlateLicense, TruckType, Ticket_no, Size_inch, InboundWeight, MillWeight, OutboundWeight, Purchase_price_thb_ton, Price_ton, Month, Year, TransactionType
Wall time: 1min 46s


# Cleansing

In [3]:
df = df1.copy()
df = df.loc[df.Size_inch=='ยูคาไม้ท่อน 2.0 นิ้ว']
df = df.loc[~df.SiteName.isin(['สามง่าม','วังวน','ตาก','บางกระทุ่ม', 'CC6'])]
df.loc[df.SiteName=='โรงสับไม้กำแพงเพชร', 'Sloc'] = 'CH11'
df.loc[df.SiteName=='โรงสับไม้ชุมพวง', 'Sloc'] = 'CH12'
df.loc[df.SiteName=='โรงสับไม้พระยืน', 'Sloc'] = 'CH13'
df['Zone'] = df['Sloc'].str[:2]
ck = ['Zone', 'Sloc', 'Year', 'Month', 'Day', 'MillWeight', 'Price_ton', 'Purchase_price_thb_ton'] # WorkCentreName, 'Time' ['SiteName','Ticket_no']
df[ck].to_csv('View_Transaction_SFT_6049_'+ts_str+'.csv', index=False)

In [ ]:
sloc = dict(df.groupby('SiteName')['Sloc'].max())
sloc = dict([(value, key) for key, value in sloc.items()])
zone = df.groupby('Zone').agg({'Sloc': lambda x: list(np.unique(x))})['Sloc'].to_dict()

# read

In [6]:
df = pd.read_csv('View_Transaction_SFT_6049_'+ts_str+'.csv', parse_dates=True)

# type

In [7]:
df.Year = df.Year.astype('str')
df.Month = df.Month.astype('str')

# Cumulative

In [ ]:
ym, ymi, ymc, ymp, ympd = {}, {}, {}, {}, {}
months = [dt.date(dt.datetime.now().year, i, 1).strftime('%b') for i in range(1, 13)]

In [ ]:
for s in ['all']+list(zone.keys())+list(sloc.keys()):
    if s == 'all':
        dfs = df
    elif len(s)==2:
        dfs = df.loc[df.Zone==s]
    else:
        dfs = df.loc[df.Sloc==s]
    ym[s], ymi[s], ymc[s] = {}, {}, {}
    for t in ['Purchase_price_thb_ton','MillWeight']:
        ym[s][t] = dfs.groupby(['Year','Month'])[t].sum().reset_index()
        ymi[s][t]= ym[s][t][ym[s][t].Year.isin(['2016', '2017', '2018', '2019', '2020'])].reset_index(drop=True)
        ymc[s][t]= ymi[s][t].groupby('Year').cumsum().merge(ymi[s][t], left_index=True,right_index=True,suffixes=('_c',''))
    ymp[s] = ymc[s]['Purchase_price_thb_ton'].merge(ymc[s]['MillWeight'])
    ymp[s]['Price']  = ymp[s]['Purchase_price_thb_ton'] / ymp[s]['MillWeight']
    ymp[s]['Price_c']= ymp[s]['Purchase_price_thb_ton_c'] / ymp[s]['MillWeight_c']
    ymp[s].Year = ymp[s].Year.astype('category')
    ymp[s].Month = ymp[s].Month.astype('category')
    
    f, forecaster = {}, {}
    print(s, df.Month.unique())
    for m in df.Month.unique():
        print(m)

        if len(ymp[s].loc[ymp[s].Month == m]) > 1:

            # 2nd-Step: PQ-Regression
            f[m] = RandomForestRegressor()
            f[m].fit(ymp[s].loc[ymp[s].Month == m].MillWeight.values.reshape(-1, 1),
                     ymp[s].loc[ymp[s].Month == m]['Price'].values)

            # 1st-Step: TS-Forecasting
            hist = pd.Series(ymp[s].loc[ymp[s].Month == m, 'MillWeight'].astype(float).values)
            print(hist)
            forecaster[m] = ThetaForecaster(sp=1)
            forecaster[m].fit(hist)
            yp = forecaster[m].predict([1]).values[0]

            if m in ymp[s].loc[ymp[s].Year == dt.datetime.now().year, 'Month']:
                ymp[s] = ymp[s].append(pd.Series({'Month': m,
                                                  'Year': str(dt.datetime.now().year + 1) + 'F',
                                                  'MillWeight': yp}), ignore_index=True)
            else:
                ymp[s] = ymp[s].append(pd.Series({'Month': m,
                                                  'Year': str(dt.datetime.now().year) + 'F',
                                                  'MillWeight': yp}), ignore_index=True)
                yp = forecaster[m].update_predict_single(pd.Series(yp), fh=[1]).values[0]
                ymp[s] = ymp[s].append(pd.Series({'Month': m,
                                                  'Year': str(dt.datetime.now().year + 1) + 'F',
                                                  'MillWeight': yp}), ignore_index=True)

                x_pred = ymp[s].loc[(ymp[s].Year == str(dt.datetime.now().year) + 'F') &
                                    (ymp[s].Month == m), 'MillWeight'].values[0]
                print(x_pred, 1, '\n')
                y_pred = f[m].predict(x_pred.reshape(1, -1))
                ymp[s].loc[ymp[s].Year == str(dt.datetime.now().year) + 'F', 'Price'] = y_pred

            x_pred = ymp[s].loc[(ymp[s].Year == str(dt.datetime.now().year + 1) + 'F') &
                                (ymp[s].Month == m), 'MillWeight'].values[0]
            print(x_pred, 2, '\n')
            y_pred = f[m].predict(x_pred.reshape(1, -1))
            ymp[s].loc[ymp[s].Year == str(dt.datetime.now().year + 1) + 'F', 'Price'] = y_pred
            print(m, m)
            
    a = ymp[s].copy()
    a['Y'] = 'Year'
    a['Type'] = 'Actual'
    a.loc[a['Year'].str.contains('F'), 'Type'] = 'Forecast'
    a['YP'] = ymp['all']['Year'].str.replace('F','').astype(int)
    b = ymp[s].copy()
    b['Y'] = 'Price'
    b['Type'] = 'Actual'
    b.loc[a['Year'].str.contains('F'), 'Type'] = 'Forecast'
    b['YP'] = ymp['all']['Price']
    ympd[s] = b.append(a, ignore_index=True)

In [ ]:
ymp['all']

In [ ]:
ympd['all']

In [ ]:
fig = px.scatter(ympd['all'], x='MillWeight', y='YP', facet_row='Y', facet_col='Month')
fig.update_yaxes(matches=None)
fig.for_each_annotation(lambda a: a.text)
#fig.for_each_trace(lambda t: t.update(name=''))
fig.show()

In [ ]:
for t in ['Purchase_price_thb_ton','MillWeight']:
    if t == 'MillWeight': cols = ['MillWeight', 'MillWeight_c']
    else: cols = ['Purchase_price_thb_ton', 'Purchase_price_thb_ton_c']
    for col in cols:
        fig = go.Figure()
        for year in ymc[s][t].Year.unique():
            Y  = ymc[s][t].loc[ymc[s][t].Year==year]
            fig.add_trace(go.Scatter(x=months, y=Y[col], mode="markers+text+lines", textposition="middle right",
                                     text=['' for i in range(len(Y)-1)]+[str(year)]))
        fig.update_layout(showlegend=False, title=col)
        fig.show()

In [ ]:
for col in ['Price', 'Price_c']:
    fig = go.Figure()
    for year in ymp[s].Year.unique():
        Y  = ymp[s].loc[ymp[s].Year==year]
        fig.add_trace(go.Scatter(x=months, y=Y[col],
                                 mode="markers+text+lines", textposition="middle right",
                                 text=['' for i in range(len(Y)-1)]+[str(year)]))
    fig.update_layout(showlegend=False, title=col)
    fig.show()

# Supply Estimation

# Export

In [ ]:
wp = {}
for s in sloc.keys():
    df_ = df.loc[df.Sloc==s].groupby(['Year', 'Month', 'Day']).sum().reset_index()
    df_['Price_Daily'] = df_['Purchase_price_thb_ton']/df_['MillWeight']
    df_.drop('Price_ton', axis=1 ,inplace=True)
    wp[s] = df_.to_json(orient = 'records')

In [ ]:
import json
with open('Sloc.json', 'w') as o: json.dump(sloc, o)
with open('Zone.json', 'w') as o: json.dump(zone, o)
with open('wp.json', 'w') as o: json.dump(wp, o)

In [ ]:
wpm = {}
with open('wp.json', 'r') as o: wp = json.load(o)
for s in sloc.keys():
    wp[s]          = pd.read_json(wp[s], orient = 'records')
    wp[s]['dt']    = pd.to_datetime(wp[s][['Year', 'Month', 'Day']])
    #wp[s].drop(['Year', 'Month', 'Day'], axis=1, inplace=True)
    wpm[s]         = wp[s].groupby(['Year','Month']).sum().reset_index().drop('Price_Daily', axis=1)
    wpm[s]['Price_Monthly'] = wpm[s]['Purchase_price_thb_ton']/wpm[s]['MillWeight']
    wpm[s]['Day']  = 1   
    wpm[s]['dt']   = pd.to_datetime(wpm[s][['Year', 'Month', 'Day']])
    wpm[s]['Sloc'] = s

# Test

In [ ]:
wp[s]

In [ ]:
wpms = {}
for s in sloc.keys():
    sen        = wpm[s][['MillWeight','Price_Monthly']].pct_change().apply(lambda x: round(x, 5))*100
    sen['sen'] = sen['MillWeight'] / sen['Price_Monthly']
    sen        = sen.replace(np.inf, 1000).replace(-np.inf, -1000).clip(-100, 100)
    wpms[s]    = sen.merge(wpm[s]['dt'], left_index=True, right_index=True)
    wpms[s]['Sloc'] = s

In [ ]:
wpmj, wpmsj = {}, {};
wpmsjz = pd.DataFrame()
for z in zone.keys():
    a, b = pd.DataFrame(), pd.DataFrame()
    for s in zone[z]:
        a = a.append(wpm[s])
        b = b.append(wpms[s])
    wpmj[z] = a
    wpmsj[z] = b
    wpmsjz = wpmsjz.append(wpmsj[z])

In [ ]:
tl = np.sort(wpmsjz.dt.unique())[::-1]
ss = wpmsjz.Sloc.unique()
dft= pd.DataFrame()
for t in tl:
    dfs = pd.DataFrame(ss, columns=['Sloc'])
    dfs['dt'] = t
    dft = dft.append(dfs)
wpmsjzm = wpmsjz.merge(dft, left_on=['dt','Sloc'], right_on=['dt','Sloc'], how='outer')


# Plot

In [ ]:
len(px.scatter(wpmj[z], x='dt', y='Price_Monthly', size='MillWeight', color='Sloc', size_max=60).data)

In [ ]:
px.line(wpms[s], x='dt', y='sen')

In [ ]:
fig = go.Scatter(x=wpm[s]['dt'].values, y=wpm[s]['Price_Monthly'].values,
                 marker=dict(size=wpm[s]['MillWeight'].values))
fig